In [24]:


!pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # Uncomment only if you have not selected the right workbench image
!pip install promptbench

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 86.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 92.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstalled transformers-4.36.2


In [25]:
import promptbench as pb
from promptbench.models import LLMModel
from promptbench.dyval import *
from promptbench.dyval.DAG.math_dag import ArithmeticDAG, ArithmeticDAGDescriber
from promptbench.dyval.DAG.logic_dag import BoolDAG, BoolDAGDescriber

#from promptbench.dyval import DyValDataset
#from promptbench.dyval import DAG



In [26]:
# main function, defines the pipeline of dyval
print(pb.dyval.DYVAL_DATASETS)
dataset_name = "arithmetic"
prompts = DYVAL_PROMPTS[dataset_name]

# create dataset and model
dataset = DyValDataset(dataset_name, 
                        is_trainset=False, # if true, it will generate a training dataset including inference steps for each problem
                        num_samples=500, # number of samples generated
                        num_nodes_per_sample=10, # number of nodes per sample, used in general DAG generation
                        min_links_per_node=1, # minimum number of links per node, used in general DAG generation
                        max_links_per_node=4, # maximum number of links per node, used in general DAG generation
                        depth=3, # depth of the DAG, used in tree DAG generation
                        num_children_per_node=2, # number of children per node, used in tree DAG generation
                        extra_links_per_node=0, # it controls if we add extra links per node to add complexity
                        add_rand_desc=0, # it controls if we add random descriptions to the nodes to add complexity
                        delete_desc=0, # it controls if we delete descriptions to the nodes, if delete, the problem is unsolvable since some nodes are not described
                        add_cycles=0, # it controls if we add cycles to the DAG, if add, the problem is unsolvable since there are loops
                        )

['arithmetic', 'linear_equation', 'bool_logic', 'deductive_logic', 'abductive_logic', 'reachability', 'max_sum_path']


  0%|          | 0/500 [00:00<?, ?it/s]


ModuleNotFoundError: No module named 'promptbench.dyval.DAG'

In [ ]:
dataset["topological"][0] # sample data

In [ ]:

print(pb.SUPPORTED_MODELS)
model = LLMModel("google/flan-t5-large", 
                max_new_tokens=10, 
                temperature=0, 
                model_dir=None,
                openai_key=None,
                sleep_time=3
                )


In [ ]:
# for each prompt, evaluate the score
for prompt in prompts:
    score = {}

    # three orders of the dataset: topological, reversed, random
    for order in ["topological"]:
    # for order in ["topological", "reversed", "random"]:
        data = dataset[order]
        preds = []
        answers = []
        
        for d in data[:1]:
            input_text = pb.InputProcess.basic_format(prompt, d)
            raw_pred = model(input_text)

            # dyval preds are processed differently, please refer to the source code /promptbench/dyval/dyval_utils.py
            pred = process_dyval_preds(raw_pred)
            preds.append(pred)
            answers.append(d["answers"])

            print(f"Input: {input_text}")
            print(f"Raw Pred: {raw_pred}")
            print(f"Pred: {pred}")
            print(f"Answer: {d['answers']}")
            print("\n")

        score[order] = dyval_evaluate(dataset.dataset_type, preds, answers)

print(score)